# TEsorter - https://github.com/zhangrengang/TEsorter


In [ ]:
# Apos ter baixado o TEsorter

## 1 - Se entra na pasta TEsorter/TEsorter/database e baixa o arquivo Pfam-A.hmm.gz
wget -c https://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz 
gunzip Pfam-A.hmm.gz # descompacta o arquivo
sudo hmmpress # cria o banco de dados do HMM para o TEsorter rodar corretamente 

## 2 Entrar no link https://isfinder.biotoul.fr/scripts/search-db.php e baixar todos os fastas de origem Streptococcus agalactiae, dentro da mesma pasta especificada no 1 passo (TEsorter/TEsorter/database)
### Para baixar os fastas de origem Streptococcus agalactiae, é necessário buscar (search) Streptococcus agalactiae e depois acessar o numero de acesso (Accession Number)

# TEsorter
## 1 - Entrar na pasta do TEsorter 
cd /mnt/dados/victor_baca/tools/TEsorter/TEsorter
### E criar um diretório para os resultados
mkdir outputs

## 2 - Encontrar o arquivo fasta do ISEScan
find path/strain/genoma -iname *.is.fna
## EX:
find /mnt/dados/victor_baca/outputs/isescan/01173/genoma -iname *.is.fna

## 3 - Rodar o TEsorter
TEsorter --db-hmm /mnt/dados/victor_baca/tools/TEsorter/TEsorter/database/Pfam-A.hmm -p 5 -cov 80 -eval 0.0001 -rule 80-80-80 path/file.fasta

## --db-hmm: banco de dados de domínios de proteínas Pfam A (arquivo .hmm) 
## -p 5: número de threads
## -cov 80: cobertura mínima do domínio
## -eval 0.0001: valor de e-valor máximo
## -rule 80-80-80: regra de classificação de TE

### EX:
TEsorter --db-hmm /mnt/dados/victor_baca/tools/TEsorter/TEsorter/database/Pfam-A.hmm -cov 80 -eval 0.0001 -rule 80-80-80 /mnt/dados/victor_baca/outputs/isescan/01173/genoma/GCF_013000945.1_ASM1300094v1_genomic.fna.is.fna


# Visualizar os resultados
## dom.tsv - valor da probabilidade de ser TE -> anotar se todos os valores são maiores que 0.8 e anotar o dominio (motivo catalítico) Pfam detectado (ex: DDE - HUH) das transposases

FILE="01173.fasta.Pfam-A.dom.tsv"; \
echo -e "\nDomínios Pfam detectados:"; \
awk -F'\t' 'NR>1 {split($1,a,"|"); print a[2]}' "$FILE" | sort | uniq -c; \
echo -e "\nVerificação de probability > 0.8:"; \
awk -F'\t' 'NR>1 {if ($5 <= 0.8) {print "ALERTA: " $1 " → " $5; count++}} END {if (count == 0) print "Todos têm probability > 0.8 ✅"}' "$FILE"